## Config

In [15]:
# Import dependencies
import tekore as tk
import os
import time
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta

# Environment variables
CLIENTID = os.environ.get('CLIENTID')
CLIENTSECRET = os.environ.get('CLIENTSECRET')
REDIRECT_URI = 'https://example.com/callback'

In [3]:
### Authenticate Tekore and Authorize App
# Get client token
app_token = tk.request_client_token(CLIENTID, CLIENTSECRET)
# Get a user token; Note the need to ask for write scope in order to create/edit playlists
# This will open a browser window; the user will need to copy the URL from the browser and paste it into the VSCode Command Palette
user_token = tk.prompt_for_user_token(CLIENTID, CLIENTSECRET, REDIRECT_URI, scope=tk.scope.write)
# Create spotify instance
spotify = tk.Spotify(app_token)
# Get the user's spotifyID; The spotify instance will have to reference the user_token to be authorized for this data
with spotify.token_as(user_token):
    userID = spotify.current_user().id

Opening browser for Spotify login...


## Get some songs and sort them

In [40]:
# Extract list of songs
songs = pd.read_csv('songs2.csv')

# Clean
# Convert to datetime
songs['albumReleaseDate'] = pd.to_datetime(songs['albumReleaseDate'])

# Sort
# Create a categorical sort order
priorPlaylists = [
    '2023',
    'Summer 2022',
    'Summer 2021',
    'Summer 2020',
    'Summer 2019',
    'Summer 2018',
    'anti pop',
    'big on the internet',
    'Lorem',
    'Alternative Hip-Hop',
    'Et-Alt'
]
songs['artistSource'] = pd.Categorical(songs['artistSource'], categories=priorPlaylists)
songs = songs.sort_values(by=['artistSource', 'artistGenreUnique', 'artistID', 'albumID'], ignore_index=True)

In [41]:
# Filter
# Date
MINDATE = datetime((date.today() - relativedelta(years=1)).year, 6, 1)
filDate = songs.albumReleaseDate>=MINDATE
# Album Type
filAType = songs.albumType=='album'
# Number of entries
lastNumber = 0
newNumber = 1000
songs = songs.loc[filDate & filAType][lastNumber:newNumber]

In [42]:
# Sort by Popularity from center
# songs['sorter'] = songs['artistPopularity'] - 50
# songs['sorter'] = songs['sorter'].abs()
# songs = songs.sort_values('sorter', ignore_index=True)

## Create an empty playlist in the user's account

In [44]:
# Create a playlist for the user
with spotify.token_as(user_token):
    newPlaylist = spotify.playlist_create(userID, "Summer Songs 1", public=False, description='Recent albums from Artist Candidates')
playlistID = newPlaylist.id

## Add the songs to the playlist

In [45]:
# Cut the df into chunks
n = 0
while n < len(songs):
    songsToAdd = songs.iloc[n:(n+100)]
    # Add songs to the playlist
    try:
        with spotify.token_as(user_token):
            additions = spotify.playlist_add(playlistID, list(songsToAdd['songURI']), position=None)
            # Wait 2 seconds so we do not exceed our API call allowance
            time.sleep(2)
    except: 
        with spotify.token_as(user_token):
            for songURI in songsToAdd['songURI']:
                try:
                    additions = spotify.playlist_add(playlistID, songURI, position=None)
                    # Wait 2 seconds so we do not exceed our API call allowance
                    time.sleep(2)
                except: 
                    pass
    n += 100